# Univariate Demand Forecasting of 911 CallS

## Libraries

In [ ]:
import pandas as pd

In [ ]:
# Read CSV Data
raw_data = pd.read_csv('CLT_FY18-24')

# (Per PM Directions. 'Other' has potentially relevant fire calls too)
# Filter for EMS
ems_calldata = raw_data[raw_data['CauseCategory'] == 'EMS'].reset_index()

# 'Dispatched' to hold value from pandas datetime func
ems_calldata['Dispatched'] = pd.to_datetime(ems_calldata['Dispatched'])

# Get Date from 'Dispatched'
ems_calldata['Date'] = ems_calldata['Dispatched'].dt.date

# Extract date info and time from 'Dispatched'
ems_calldata['Date'] = ems_calldata['Dispatched'].dt.date
ems_calldata['Year'] = ems_calldata['Dispatched'].dt.year
ems_calldata['Month'] = ems_calldata['Dispatched'].dt.month
ems_calldata['Day'] = ems_calldata['Dispatched'].dt.day
ems_calldata['Time'] = ems_calldata['Dispatched'].dt.time

# Sort by 'Dispatched'
ems_calldata = ems_calldata.sort_values('datetime')

# Organize by 'Dispatched' (per hour) and lat&lon
ems_sorted_data = ems_calldata.groupby(['Latitude', 'Longitude', 'Time']).size().reset_index(name='Demand')

# Call frequency by date
freq_count = ems_calldata.groupby('Date').size().reset_index(name='Count')

freq_count


## Model Building - LTSM

In [ ]:
ems_calldata.set_index('Date', inplace=True)

# Adding features
def add_features(calldata, lags):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data['Demand']